In [1]:
from sys import path
if '..' not in path:
    path.insert(0, '..')

In [2]:
from os import path, makedirs, environ
import numpy as np
import random
import json
from datetime import datetime

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import _library.lstm_utils as lstm_utils
from tensorflow.config import list_physical_devices
from tensorflow.config.experimental import set_memory_growth

In [4]:
from _library.utils import SYSTEM_NAMES, SUBFOLDERS, load_datasets
from tensorflow.random import set_seed as tf_set_seed
from string import ascii_uppercase 
from collections import Counter
from re import match, findall
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model  
from tensorflow.keras import metrics as metrics
from sklearn.ensemble import ExtraTreesClassifier
from datetime import datetime

#### import the validation metrics df

In [5]:
%cd /mnt/data/vieri/projects/SAMPLE/

/mnt/data/vieri/projects/SAMPLE


In [6]:
print(SYSTEM_NAMES, "\nSUBFOLDERS: -->", SUBFOLDERS)
# --- 0 ---------- 1 ---------- 2 --------- 3 ---------- 4 -------

['Binetto 1', 'Binetto 2', 'Soleto 1', 'Soleto 2', 'Galatina'] 
SUBFOLDERS: --> ['Cleaned', '1-hour sampling', '1-hour averaged sampling', 'Residuals', 'Residuals_analytical', 'Failure events', None]


In [7]:
system_name = SYSTEM_NAMES[2]

system_path = path.join('data', system_name.upper(), system_name.upper())
print(f"PV SYSTEM --> {system_name}")

PV SYSTEM --> Soleto 1


In [8]:
prefix_inv_name = 'INV'

In [9]:
num_inverters = 4 if (system_name != 'Soleto 2') else 2
inv_names = [prefix_inv_name + str(inv_num) for inv_num in range(1, num_inverters + 1)]

In [10]:
inv_names

['INV1', 'INV2', 'INV3', 'INV4']

In [11]:
num_hid_layers = 3

In [12]:
main_saving_folder = 'UC2 - LSTM'
saving_path = path.join(system_path, main_saving_folder)

In [13]:
version = " - Generalized Version"
subfolders = ['Grid search' + version, 'Trained models' + version, 'Test metrics' + version]

In [14]:
final_df = pd.DataFrame()
for inv_name in inv_names:
    best_perfomance_df_name = f'lstm_gridSearch_df_perfomance_{inv_name}_hid_layers_{num_hid_layers}.csv'
    test_df = pd.read_csv(path.join(saving_path, subfolders[0], best_perfomance_df_name))
    #classi Soleto 1
    if system_name == SYSTEM_NAMES[2]:
        test_df['f1 score a'] = 2 * (test_df['Val precision a']*test_df['Val recall a'])/(test_df['Val precision a']+test_df['Val recall a'])  

    if system_name == SYSTEM_NAMES[-1]:
        test_df['f1 score a'] = 2 * (test_df['Val precision a']*test_df['Val recall a'])/(test_df['Val precision a']+test_df['Val recall a'])
        test_df['f1 score d'] = 2 * (test_df['Val precision d']*test_df['Val recall d'])/(test_df['Val precision d']+test_df['Val recall d'])
        
    test_df['f1 score b'] = 2 * (test_df['Val precision b']*test_df['Val recall b'])/(test_df['Val precision b']+test_df['Val recall b'])
    if system_name == SYSTEM_NAMES[2]:
            test_df['f1 score ab'] = test_df['f1 score a']*test_df['f1 score b']

    test_df = test_df.sort_values(by=['f1 score b'], ascending = False).drop(columns = ['Unnamed: 0'])
    test_df['INV'] = inv_name
    final_df = final_df.append(test_df, ignore_index=True)

### test

In [22]:
inv_name = inv_names[0]

best_perfomance_df_name = f'lstm_gridSearch_df_perfomance_{inv_name}_hid_layers_{num_hid_layers}.csv'
test_df = pd.read_csv(path.join(saving_path, subfolders[0], best_perfomance_df_name))

In [38]:
test_df

,Unnamed: 0,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Precision c,...,Val precision b,Val recall b,Val precision c,Val recall c,Val precision d,Val recall d,Val binary accuracy,best_epoch,Starting neurons,Window dimensions
0,0,0.0830,0.2176,0.8569,0.8845,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.8512,0.9304,0.9658,29.0,16.0,12.0
1,1,0.0783,0.2202,0.8600,0.9027,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.8622,0.9148,0.9659,35.0,32.0,12.0
2,2,0.0681,0.2241,0.8727,0.9214,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.8676,0.9578,0.9722,45.0,64.0,12.0
3,3,0.0667,0.2237,0.8697,0.9214,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.8647,0.9547,0.9713,46.0,92.0,12.0
4,4,0.0643,0.2250,0.8791,0.9218,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.8645,0.9679,0.9729,54.0,128.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,58,0.0810,0.2187,0.9079,0.8443,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.9228,0.9065,0.9752,8.0,64.0,264.0
59,59,0.0586,0.2288,0.9357,0.8957,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.9659,0.9058,0.9815,10.0,92.0,264.0
60,60,0.0890,0.2150,0.9293,0.8002,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.9247,0.9026,0.9750,2.0,128.0,264.0
61,61,0.0627,0.2273,0.9283,0.8908,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.9815,0.8890,0.9813,5.0,192.0,264.0


In [104]:
test_df['f1 score b'] = 2 * (test_df['Val precision b']*test_df['Val recall b'])/(test_df['Val precision b']+test_df['Val recall b'])

In [105]:
test_df = test_df.sort_values(by=['f1 score b'], ascending = False).drop(columns = ['Unnamed: 0'])

In [106]:
inv_name = inv_names[1]

best_perfomance_df_name = f'lstm_gridSearch_df_perfomance_{inv_name}_hid_layers_{num_hid_layers}.csv'
test_df2 = pd.read_csv(path.join(saving_path, subfolders[0], best_perfomance_df_name))

In [107]:
test_df2['f1 score b'] = 2 * (test_df2['Val precision b']*test_df2['Val recall b'])/(test_df2['Val precision b']+test_df2['Val recall b'])

In [108]:
test_df2 = test_df2.sort_values(by=['f1 score b'], ascending = False).drop(columns = ['Unnamed: 0'])

In [109]:
test_df['INV'] = 1

In [110]:
test_df2['INV'] = 2

In [111]:
final_df = test_df.append(test_df2, ignore_index=True)

In [112]:
final_df

,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Binary accuracy,Val loss,...,Val precision a,Val recall a,Val precision b,Val recall b,Val binary accuracy,best_epoch,Starting neurons,Window dimensions,f1 score b,INV
0,0.0122,0.4952,0.9933,0.9867,0.0,0.00,0.9933,0.9873,0.9961,0.0065,...,0.0,0.0,0.9973,0.9959,0.9987,40.0,32.0,72.0,0.996600,1
1,0.0131,0.6951,0.9950,0.9850,1.0,0.25,0.9950,0.9855,0.9961,0.0072,...,0.0,0.0,1.0000,0.9925,0.9985,39.0,128.0,72.0,0.996236,1
2,0.0144,0.4947,0.9953,0.9830,0.0,0.00,0.9953,0.9835,0.9958,0.0086,...,0.0,0.0,0.9986,0.9924,0.9981,48.0,128.0,168.0,0.995490,1
3,0.0116,0.4952,0.9959,0.9845,0.0,0.00,0.9959,0.9850,0.9962,0.0083,...,0.0,0.0,0.9958,0.9944,0.9980,41.0,92.0,168.0,0.995100,1
4,0.0161,0.4947,0.9940,0.9842,0.0,0.00,0.9940,0.9847,0.9957,0.0128,...,0.0,0.0,0.9958,0.9937,0.9979,19.0,64.0,264.0,0.994749,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0.1107,0.4014,0.9337,0.7042,0.0,0.00,0.9337,0.7042,0.9491,0.0990,...,0.0,0.0,0.9480,0.7491,0.9575,37.0,64.0,12.0,0.836894,2
122,0.1127,0.4012,0.9414,0.6993,0.0,0.00,0.9414,0.6993,0.9494,0.1001,...,0.0,0.0,0.9529,0.7381,0.9566,30.0,32.0,12.0,0.831857,2
123,0.1158,0.3985,0.9321,0.6961,0.0,0.00,0.9321,0.6961,0.9478,0.0993,...,0.0,0.0,0.9560,0.7336,0.9564,27.0,256.0,12.0,0.830163,2
124,0.1170,0.3995,0.9375,0.6961,0.0,0.00,0.9375,0.6961,0.9485,0.1062,...,0.0,0.0,0.9123,0.7591,0.9544,31.0,16.0,12.0,0.828679,2


In [ ]:
final_df['f1 score b'] = 2 * (test_df2['Val precision b']*test_df2['Val recall b'])/(test_df2['Val precision b']+test_df2['Val recall b'])

### end test

In [15]:
final_df

,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Precision c,Recall c,...,Val precision c,Val recall c,Val binary accuracy,best_epoch,Starting neurons,Window dimensions,f1 score a,f1 score b,f1 score ab,INV
0,0.0143,0.6157,0.9111,0.8356,0.9022,0.9282,0.9196,0.9449,0.0,0.0000,...,0.0,0.0000,0.9969,30.0,64.0,48.0,0.954128,0.977776,0.932924,INV1
1,0.0197,0.5950,0.8693,0.8181,0.8552,0.9167,0.8833,0.9174,0.0,0.0000,...,0.0,0.0000,0.9970,16.0,16.0,48.0,0.958491,0.977580,0.937002,INV1
2,0.0126,0.6415,0.9586,0.8435,0.9512,0.9435,0.9656,0.9456,1.0,0.0109,...,1.0,0.0714,0.9976,62.0,256.0,12.0,0.951870,0.974374,0.927478,INV1
3,0.0154,0.6071,0.8957,0.8255,0.8904,0.9242,0.9008,0.9282,0.0,0.0000,...,0.0,0.0000,0.9971,29.0,32.0,96.0,0.957442,0.973860,0.932415,INV1
4,0.0163,0.6140,0.9072,0.8344,0.9050,0.9310,0.9093,0.9394,0.0,0.0000,...,0.0,0.0000,0.9966,10.0,192.0,48.0,0.945450,0.973675,0.920561,INV1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0.0150,0.3897,0.7669,0.5507,0.5934,0.2903,0.8340,0.7313,0.0,0.0000,...,0.0,0.0000,0.9935,7.0,16.0,120.0,0.531933,0.824201,0.438420,INV4
248,0.0145,0.4125,0.6675,0.6228,0.5484,0.4570,0.7376,0.7405,0.0,0.0000,...,0.0,0.0000,0.9943,9.0,92.0,168.0,0.660187,0.802680,0.529920,INV4
249,0.0160,0.2514,0.7381,0.3002,0.6176,0.1207,0.7687,0.4310,0.0,0.0000,...,0.0,0.0000,0.9942,4.0,64.0,264.0,0.530108,0.796870,0.422427,INV4
250,0.0125,0.4653,0.7253,0.7475,0.5397,0.5340,0.8360,0.8833,0.0,0.0000,...,0.0,0.0000,0.9935,17.0,32.0,24.0,0.494130,0.780118,0.385480,INV4


In [16]:
val_metrics_df = final_df.groupby(by = ['Starting neurons','Window dimensions']).sum()

In [17]:
val_metrics_df

Loss  F1 score  Precision  Recall  \
Starting neurons Window dimensions                                        
16.0             12.0               0.0901    1.3390     2.3298  2.2690   
                 24.0               0.0837    1.3251     2.2997  2.3148   
                 48.0               0.0840    1.3190     2.2613  2.3048   
                 72.0               0.1432    1.1228     1.6550  1.6644   
                 96.0               0.0820    1.3161     2.3349  2.2731   
...                                    ...       ...        ...     ...   
256.0            96.0               0.0471    1.4265     2.4923  2.4846   
                 120.0              0.0508    1.2834     2.3106  2.1792   
                 168.0              0.0435    1.4075     2.4592  2.4226   
                 216.0              0.0487    1.3359     2.3558  2.3323   
                 264.0              0.0464    1.4039     2.4430  2.4691   

                                    Precision a  Recall a  Precision b  \
Starting neurons Window dimensions                                       
16.0             12.0                    2.1559    2.2830       1.8347   
                 24.0                    2.1260    2.2782       1.7748   
                 48.0                    2.0847    2.2710       1.7632   
                 72.0                    1.4992    1.6306       1.7699   
                 96.0                    2.1901    2.2163       1.7484   
...                                         ...       ...          ...   
256.0            96.0                    2.4069    2.4713       1.8245   
                 120.0                   2.1382    2.1227       1.7327   
                 168.0                   2.3548    2.4146       1.7954   
                 216.0                   2.2213    2.3094       1.7469   
                 264.0                   2.3178    2.4959       1.7970   

                                    Recall b  Precision c  Recall c  ...  \
Starting neurons Window dimensions                                   ...   
16.0             12.0                 1.7727          0.0    0.0000  ...   
                 24.0                 1.7788          0.0    0.0000  ...   
                 48.0                 1.8035          0.0    0.0000  ...   
                 72.0                 1.8481          0.0    0.0000  ...   
                 96.0                 1.7467          0.0    0.0000  ...   
...                                      ...          ...       ...  ...   
256.0            96.0                 1.8605          0.0    0.0000  ...   
                 120.0                1.7107          0.0    0.0000  ...   
                 168.0                1.8358          1.0    0.0116  ...   
                 216.0                1.7460          0.0    0.0000  ...   
                 264.0                1.8214          0.0    0.0000  ...   

                                    Val recall a  Val precision b  \
Starting neurons Window dimensions                                  
16.0             12.0                     2.6279           1.8261   
                 24.0                     2.4392           1.7580   
                 48.0                     2.6091           1.8574   
                 72.0                     1.6618           1.9392   
                 96.0                     2.7183           1.8949   
...                                          ...              ...   
256.0            96.0                     2.7354           1.9015   
                 120.0                    2.5914           1.7389   
                 168.0                    2.7448           1.8757   
                 216.0                    2.5086           1.8381   
                 264.0                    2.5790           1.8607   

                                    Val recall b  Val precision c  \
Starting neurons Window dimensions                                  
16.0             12.0                     1.8983              0.0   
     

#### select the best configuration among all the plants

In [18]:
val_metrics_df.sort_values(by=['f1 score ab'], ascending = False)
#val_metrics_df.sort_values(by=['f1 score d'], ascending = False)
#val_metrics_df.sort_values(by=['f1 score b'], ascending = False)

,,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Precision c,Recall c,...,Val recall a,Val precision b,Val recall b,Val precision c,Val recall c,Val binary accuracy,best_epoch,f1 score a,f1 score b,f1 score ab
Starting neurons,Window dimensions,,,,,,,,,,,,,,,,,,,,,
192.0,72.0,0.0444,1.4171,2.4641,2.4407,2.3902,2.4155,1.8263,1.8794,0.0,0.0,...,2.7520,1.9286,1.9390,0.0,0.0000,3.9919,75.0,2.544001,1.933772,1.697582
256.0,96.0,0.0471,1.4265,2.4923,2.4846,2.4069,2.4713,1.8245,1.8605,0.0,0.0,...,2.7354,1.9015,1.9615,0.0,0.0000,3.9919,61.0,2.547029,1.930572,1.679160
128.0,72.0,0.0514,1.4248,2.4851,2.4743,2.3963,2.4497,1.8376,1.8711,0.0,0.0,...,2.7314,1.8705,1.9390,1.0,0.0455,3.9916,105.0,2.568655,1.903508,1.677212
92.0,48.0,0.0529,1.4426,2.5236,2.4571,2.4380,2.4457,1.8874,1.8899,0.0,0.0,...,2.5744,1.9126,1.8989,0.0,0.0000,3.9905,94.0,2.517832,1.903941,1.676987
16.0,216.0,0.0781,1.3519,2.3758,2.2481,2.2569,2.2599,1.7828,1.8191,0.0,0.0,...,2.7469,1.9405,1.9367,0.0,0.0000,3.9935,113.0,2.574363,1.937404,1.668780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92.0,168.0,0.0643,1.2922,2.3411,2.2722,2.2188,2.2139,1.6498,1.6820,0.0,0.0,...,2.6129,1.6912,1.8120,0.0,0.0000,3.9885,60.0,2.399073,1.747080,1.407645
64.0,264.0,0.0738,1.0987,2.3528,1.8621,2.2234,1.7966,1.6552,1.3485,0.0,0.0,...,2.1976,1.7165,1.7881,0.0,0.0000,3.9886,39.0,2.219321,1.751396,1.342018
16.0,120.0,0.0878,1.2331,2.3320,2.1096,2.1574,1.9582,1.7362,1.6621,0.0,0.0,...,2.4079,1.6723,1.8847,0.0,0.0000,3.9873,71.0,2.269866,1.771593,1.324534


In [23]:
final_df[final_df['Window dimensions'] == 72]

,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Precision c,Recall c,...,Val precision c,Val recall c,Val binary accuracy,best_epoch,Starting neurons,Window dimensions,f1 score a,f1 score b,f1 score ab,INV
6,0.0126,0.6244,0.9168,0.8578,0.9137,0.9576,0.9197,0.9569,0.0,0.0,...,0.0,0.0000,0.9971,29.0,192.0,72.0,0.966700,0.973272,0.940862,INV1
17,0.0136,0.6175,0.9163,0.8393,0.9091,0.9322,0.9233,0.9407,0.0,0.0,...,0.0,0.0000,0.9969,27.0,64.0,72.0,0.961320,0.968423,0.930964,INV1
18,0.0124,0.6264,0.9240,0.8566,0.9210,0.9548,0.9269,0.9569,0.0,0.0,...,0.0,0.0000,0.9972,29.0,256.0,72.0,0.978032,0.968090,0.946823,INV1
22,0.0162,0.6139,0.9098,0.8356,0.8992,0.9322,0.9202,0.9326,0.0,0.0,...,0.0,0.0000,0.9966,26.0,16.0,72.0,0.949720,0.962172,0.913794,INV1
30,0.0163,0.6114,0.9025,0.8356,0.8962,0.9266,0.9086,0.9380,0.0,0.0,...,0.0,0.0000,0.9965,12.0,92.0,72.0,0.945086,0.957000,0.904447,INV1
36,0.0178,0.6069,0.8905,0.8344,0.8830,0.9379,0.8979,0.9245,0.0,0.0,...,0.0,0.0000,0.9962,14.0,32.0,72.0,0.931818,0.950844,0.886014,INV1
46,0.0118,0.6170,0.9162,0.8381,0.9116,0.9322,0.9206,0.9380,0.0,0.0,...,1.0,0.0455,0.9960,43.0,128.0,72.0,0.929770,0.943008,0.876780,INV1
84,0.0688,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,...,0.0,0.0000,0.9943,1.0,16.0,72.0,NaN,NaN,NaN,INV2
85,0.0070,0.2231,0.6480,0.6923,0.6480,0.6923,0.0000,0.0000,0.0,0.0,...,0.0,0.0000,0.9975,25.0,32.0,72.0,0.805536,NaN,NaN,INV2
86,0.0065,0.2381,0.6836,0.7479,0.6836,0.7479,0.0000,0.0000,0.0,0.0,...,0.0,0.0000,0.9971,25.0,64.0,72.0,0.759108,NaN,NaN,INV2


In [27]:
final_df[final_df.INV == 'INV1'].sort_values(by='F1 score',ascending = False)

,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Precision c,Recall c,...,Val precision c,Val recall c,Val binary accuracy,best_epoch,Starting neurons,Window dimensions,f1 score a,f1 score b,f1 score ab,INV
2,0.0126,0.6415,0.9586,0.8435,0.9512,0.9435,0.9656,0.9456,1.0,0.0109,...,1.0,0.0714,0.9976,62.0,256.0,12.0,0.951870,0.974374,0.927478,INV1
37,0.0113,0.6367,0.9368,0.8437,0.9339,0.9511,0.9393,0.9503,1.0,0.0116,...,1.0,0.0500,0.9965,38.0,256.0,168.0,0.927712,0.950321,0.881624,INV1
43,0.0106,0.6354,0.9438,0.8616,0.9390,0.9620,0.9509,0.9608,0.0,0.0000,...,1.0,0.1053,0.9966,50.0,256.0,24.0,0.944702,0.947368,0.894981,INV1
54,0.0108,0.6349,0.9450,0.8604,0.9335,0.9538,0.9561,0.9661,0.0,0.0000,...,1.0,0.1053,0.9959,47.0,192.0,24.0,0.920802,0.937794,0.863523,INV1
13,0.0125,0.6338,0.9257,0.8508,0.9288,0.9504,0.9227,0.9558,1.0,0.0116,...,0.0,0.0000,0.9971,36.0,128.0,96.0,0.963341,0.969072,0.933547,INV1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0.0185,0.5996,0.8914,0.8091,0.8832,0.9038,0.8992,0.9116,0.0,0.0000,...,0.0,0.0000,0.9970,11.0,256.0,96.0,0.957874,0.969072,0.928249,INV1
27,0.0175,0.5985,0.8809,0.7972,0.8803,0.9097,0.8815,0.9206,0.0,0.0000,...,0.0,0.0000,0.9974,9.0,92.0,264.0,0.957565,0.959968,0.919232,INV1
33,0.0169,0.5974,0.8793,0.7957,0.8770,0.9064,0.8815,0.9206,0.0,0.0000,...,0.0,0.0000,0.9973,9.0,192.0,264.0,0.957565,0.954000,0.913517,INV1
32,0.0187,0.5969,0.8779,0.7957,0.8690,0.9097,0.8865,0.9175,0.0,0.0000,...,0.0,0.0000,0.9974,9.0,64.0,264.0,0.963400,0.954527,0.919591,INV1


In [28]:
final_df[final_df.INV == 'INV2'].sort_values(by='F1 score',ascending = False)

,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Precision c,Recall c,...,Val precision c,Val recall c,Val binary accuracy,best_epoch,Starting neurons,Window dimensions,f1 score a,f1 score b,f1 score ab,INV
107,0.0052,0.2647,0.7664,0.8238,0.7664,0.8238,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9976,34.0,64.0,168.0,0.780522,NaN,NaN,INV2
108,0.0053,0.2630,0.7645,0.8150,0.7645,0.8150,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9979,22.0,92.0,168.0,0.809486,NaN,NaN,INV2
125,0.0045,0.2629,0.7467,0.8358,0.7467,0.8358,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9976,36.0,256.0,264.0,0.783963,NaN,NaN,INV2
123,0.0048,0.2600,0.7432,0.8209,0.7432,0.8209,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9974,29.0,128.0,264.0,0.775191,NaN,NaN,INV2
124,0.0047,0.2546,0.7339,0.7960,0.7339,0.7960,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9973,29.0,192.0,264.0,0.765614,NaN,NaN,INV2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0.0072,0.2129,0.6360,0.6416,0.6360,0.6416,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9967,25.0,32.0,48.0,0.781045,NaN,NaN,INV2
69,0.0088,0.2058,0.6008,0.6352,0.6008,0.6352,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9972,30.0,256.0,12.0,0.757536,NaN,NaN,INV2
119,0.0659,0.0028,0.0079,0.0050,0.0256,0.0050,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9948,1.0,16.0,264.0,NaN,NaN,NaN,INV2
87,0.0522,0.0024,0.0080,0.0043,0.0213,0.0043,0.0,0.0,0.0,0.0,...,0.0,0.0,0.9943,1.0,92.0,72.0,NaN,NaN,NaN,INV2


In [29]:
final_df[final_df.INV == 'INV4'].sort_values(by='F1 score',ascending = False)

,Loss,F1 score,Precision,Recall,Precision a,Recall a,Precision b,Recall b,Precision c,Recall c,...,Val precision c,Val recall c,Val binary accuracy,best_epoch,Starting neurons,Window dimensions,f1 score a,f1 score b,f1 score ab,INV
208,0.0060,0.5798,0.9093,0.8680,0.8253,0.7569,0.9600,0.9395,0.0,0.0,...,0.0,0.0,0.9972,36.0,92.0,48.0,0.807653,0.930266,0.751332,INV4
190,0.0059,0.5738,0.8772,0.8772,0.8000,0.7692,0.9253,0.9489,0.0,0.0,...,0.0,0.0,0.9974,27.0,256.0,96.0,0.780979,0.961500,0.750911,INV4
212,0.0064,0.5646,0.8483,0.8744,0.7552,0.7796,0.9130,0.9403,0.0,0.0,...,0.0,0.0,0.9966,27.0,128.0,120.0,0.734700,0.921100,0.676732,INV4
234,0.0070,0.5635,0.8571,0.8676,0.7807,0.7644,0.9032,0.9333,0.0,0.0,...,0.0,0.0,0.9965,33.0,92.0,24.0,0.712635,0.897976,0.639929,INV4
192,0.0063,0.5630,0.8565,0.8640,0.7765,0.7637,0.9075,0.9307,0.0,0.0,...,0.0,0.0,0.9973,34.0,92.0,96.0,0.778747,0.961037,0.748405,INV4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,0.0133,0.3929,0.6880,0.5704,0.5360,0.3722,0.7692,0.7115,0.0,0.0,...,0.0,0.0,0.9960,5.0,128.0,216.0,0.641998,0.880566,0.565322,INV4
247,0.0150,0.3897,0.7669,0.5507,0.5934,0.2903,0.8340,0.7313,0.0,0.0,...,0.0,0.0,0.9935,7.0,16.0,120.0,0.531933,0.824201,0.438420,INV4
231,0.0165,0.3852,0.6687,0.5327,0.5769,0.4310,0.7286,0.6067,0.0,0.0,...,0.0,0.0,0.9964,16.0,192.0,264.0,0.707100,0.904329,0.639451,INV4
249,0.0160,0.2514,0.7381,0.3002,0.6176,0.1207,0.7687,0.4310,0.0,0.0,...,0.0,0.0,0.9942,4.0,64.0,264.0,0.530108,0.796870,0.422427,INV4
